In [2]:
import os, getpass
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.graph import START, StateGraph, MessagesState
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver

def check_tyre_wear(lap_number: int) -> str:
    """Checks the estimated tyre wear based on the current lap number."""
    if lap_number < 15:
        return f"Lap {lap_number}: Tyre wear is low. Estimated 85% life remaining."
    elif lap_number < 30:
        return f"Lap {lap_number}: Tyre wear is medium. Estimated 50% life remaining."
    else:
        return f"Lap {lap_number}: Tyre wear is high. Estimated 20% life remaining. Consider pitting."

def get_weather_forecast(sector: int) -> str:
    """Gets the weather forecast for a specific sector of the track."""
    if sector == 1:
        return f"Sector {sector}: Forecast is clear, track temperature stable."
    elif sector == 2:
        return f"Sector {sector}: Slight chance of light drizzle in the next 10 laps."
    else:
        return f"Sector {sector}: Track remains dry for now."

def suggest_pit_strategy(current_lap: int, laps_remaining: int) -> str:
    """Suggests a pit stop strategy based on current lap and laps remaining."""
    optimal_pit_window_start = 20
    optimal_pit_window_end = 35
    if optimal_pit_window_start <= current_lap <= optimal_pit_window_end:
        return f"Lap {current_lap}/{laps_remaining} laps remaining: Currently within the optimal pit window. Evaluate traffic."
    elif current_lap < optimal_pit_window_start:
        return f"Lap {current_lap}/{laps_remaining} laps remaining: Too early for optimal one-stop. Hold position."
    else:
        return f"Lap {current_lap}/{laps_remaining} laps remaining: Past optimal window. Consider Plan B if tyres degrade rapidly."


tools = [check_tyre_wear, get_weather_forecast, suggest_pit_strategy]
llm = ChatOpenAI(model="gpt-4.1-nano")
llm_with_tools = llm.bind_tools(tools)

sys_msg = SystemMessage(content="You are a helpful F1 race engineer providing concise updates and strategy advice to the driver based on available tools.")

def race_engineer_assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

builder = StateGraph(MessagesState)

builder.add_node("assistant", race_engineer_assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = builder.compile(interrupt_before=["tools"], checkpointer=memory)

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [5]:
initial_input = {"messages": [HumanMessage(content="What's the tyre wear like on lap 25?")]}
thread = {"configurable": {"thread_id": "f1_thread_1"}}

for event in graph.stream(initial_input, thread, stream_mode="values"):
    if "messages" in event:
      event['messages'][-1].pretty_print()

================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_AYjV7DyVpP1cd5seOxT6CeDA)
 Call ID: call_AYjV7DyVpP1cd5seOxT6CeDA
  Args:
    a: 2
    b: 3


In [6]:
state = graph.get_state(thread)
state.next

('tools',)

In [ ]:
for event in graph.stream(None, thread, stream_mode="values"):
    if "messages" in event:
      event['messages'][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  multiply (call_AYjV7DyVpP1cd5seOxT6CeDA)
 Call ID: call_AYjV7DyVpP1cd5seOxT6CeDA
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6


In [9]:
initial_input = {"messages": [HumanMessage(content="Any rain expected in sector 2?")]}
thread = {"configurable": {"thread_id": "f1_thread_2"}}

for event in graph.stream(initial_input, thread, stream_mode="values"):
    if "messages" in event:
      event['messages'][-1].pretty_print()

user_approval = input("Approve tool call? (yes/no): ")

if user_approval.lower() == "yes":
    for event in graph.stream(None, thread, stream_mode="values"):
         if "messages" in event:
            event['messages'][-1].pretty_print()
else:
    print("Tool call cancelled by user.")

================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_tpHvTmsHSjSpYnymzdx553SU)
 Call ID: call_tpHvTmsHSjSpYnymzdx553SU
  Args:
    a: 2
    b: 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_tpHvTmsHSjSpYnymzdx553SU)
 Call ID: call_tpHvTmsHSjSpYnymzdx553SU
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6
================================== Ai Message ==================================

The result of multiplying 2 and 3 is 6.


In [1]:
if 'google.colab' in str(get_ipython()):
    raise Exception("Unfortunately LangGraph Studio is currently not supported on Google Colab")

In [2]:
# This is the URL of the local development server
from langgraph_sdk import get_client
client = get_client(url="http://127.0.0.1:2024")

In [10]:
initial_input = {"messages": [HumanMessage(content="Suggest a pit strategy. Current lap 22, 35 laps remaining.")]}
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="f1_race_data", # Make sure this matches your langgraph.json key
    input=initial_input,
    stream_mode="values",
    interrupt_before=["tools"],
):
    print(f"Receiving new event of type: {chunk.event}...")
    messages = chunk.data.get('messages', [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': 'Multiply 2 and 3', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '2a3b1e7a-f6d9-44c2-a4b4-b7f67aa3691c', 'example': False}
--------------------------------------------------
Receiving new event of type: values...
{'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'call_ElnkVOf1H80dlwZLqO0PQTwS', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 18, 'prompt_tokens': 134, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 't

In [11]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    "f1_race_data", # Make sure this matches your langgraph.json key
    input=None,
    stream_mode="values",
    interrupt_before=["tools"], # Can keep or remove interrupt for continuation
):
    print(f"Receiving new event of type: {chunk.event}...")
    messages = chunk.data.get('messages', [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'call_ElnkVOf1H80dlwZLqO0PQTwS', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 18, 'prompt_tokens': 134, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'tool_calls', 'logprobs': None}, 'type': 'ai', 'name': None, 'id': 'run-89ee14dc-5f46-4dd9-91d9-e922c4a23572-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_ElnkVOf1H80dlwZLqO0PQTwS', 'type': 'tool_call'}], 'invalid_tool_